# 📚 DATOS EDA: Cultura y Ocio

### Unificar datos como inicio

In [ ]:
import pandas as pd

# Lista de archivos a combinar
archivos_excel = ['../data/datasets/ingresos.csv', '../data/datasets/caracteristicas.csv','../data/datasets/bienesyservicios.csv', '../data/datasets/libros.csv', '../data/datasets/espectaculos.csv', '../data/datasets/educacion.csv', '../data/datasets/renta.csv']  # Reemplaza con los nombres de tus archivos

# Diccionario para almacenar cada archivo como una hoja
hojas = {}

# Iterar sobre los archivos y cargarlos en el diccionario
for archivo in archivos_excel:
    # Cargar el archivo en un DataFrame
    df = pd.read_csv(archivo, sep=";", encoding="latin1")
    
    # Usar el nombre del archivo (sin la extensión) como el nombre de la hoja
    nombre_hoja = archivo.split('/')[-1].replace('.csv', '')
    
    # Guardar el DataFrame en el diccionario con el nombre de la hoja
    hojas[nombre_hoja] = df

# Guardar todo en un solo archivo Excel con varias hojas
with pd.ExcelWriter('../data/datos.xlsx') as writer:
    for nombre_hoja, df in hojas.items():
        df.to_excel(writer, sheet_name=nombre_hoja,index=False)

### Cargar las hojas y tratar datos

In [ ]:
# Cargar el Excel
archivo = "../data/datos.xlsx"
xlsx = pd.ExcelFile(archivo)

# Cargar cada hoja
libros = xlsx.parse("libros")
espectaculos = xlsx.parse("espectaculos")
educacion = xlsx.parse("educacion")
bys = xlsx.parse("bienesyservicios")
renta = xlsx.parse("renta")

#diferentes
ingresos = xlsx.parse("ingresos")
caracteristicas = xlsx.parse("caracteristicas")

#1) Unificar por Año y CCAA 
# Nos aseguramos de que tengan las columnas comunes con el mismo nombre
#Hay que tratar educacion pues los años se dividen en trimestres y la columna de CCAA también es diferente
educacion.rename(columns={'Periodo': 'Año'}, inplace=True)
educacion['periodo'] = educacion['Año'].str.extract(r'(\d{4})')  # Extrae los 4 primeros dígitos

educacion.rename(columns={'Comunidades y Ciudades Autónomas': 'Comunidad autónoma'}, inplace=True)
comunidad_sin_num = []

for texto in educacion['Comunidad autónoma']:
    partes = texto.split(" ", 1)  
    comunidad_sin_num.append(partes[1])  


educacion['Comunidad autónoma'] = comunidad_sin_num


## 📖 Libros

In [3]:
libros.rename(columns={'Indicador': 'Indicador_libros', 'Grupo de gasto': 'Grupo de gasto_libros', 'Total': 'Total_libros'}, inplace=True)
libros = libros[['periodo','Comunidad autónoma', 'Grupo de gasto_libros', 'Indicador_libros', 'Total_libros']]

In [4]:
libros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   periodo                1152 non-null   int64 
 1   Comunidad autónoma     1152 non-null   object
 2   Grupo de gasto_libros  1152 non-null   object
 3   Indicador_libros       1152 non-null   object
 4   Total_libros           1152 non-null   object
dtypes: int64(1), object(4)
memory usage: 45.1+ KB


In [5]:
libros.iloc[272]

periodo                                       2023
Comunidad autónoma                 Ceuta y Melilla
Grupo de gasto_libros                        Libro
Indicador_libros         TOTAL (Millones de euros)
Total_libros                                    ..
Name: 272, dtype: object

In [6]:
# Reemplazar comas por puntos
libros["Total_libros"] = libros["Total_libros"].str.replace(',', '.')

# Convertir a numérico (y evitar errores cuando hay valores como '..' ) 
# Obtendremos NaN pero luego se eliminará Ceuta y Melilla
libros["Total_libros"] = pd.to_numeric(libros["Total_libros"], errors="coerce")

In [7]:
libros.info() #Las columnas son del tipo correspondiente

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   periodo                1152 non-null   int64  
 1   Comunidad autónoma     1152 non-null   object 
 2   Grupo de gasto_libros  1152 non-null   object 
 3   Indicador_libros       1152 non-null   object 
 4   Total_libros           1088 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 45.1+ KB


In [8]:
libros["periodo"].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016])

In [9]:
libros = libros[libros["periodo"] >= 2021]

Cardinalidad

In [10]:
cardinalidad_por_columna = pd.Series()
for columna in libros:
    cardinalidad_por_columna[columna] = round(libros[columna].nunique(dropna=False) / len(libros),2)*100

In [11]:
cardinalidad_por_columna #ninguna columna es un buen índice

periodo                   1.0
Comunidad autónoma        4.0
Grupo de gasto_libros     0.0
Indicador_libros          1.0
Total_libros             75.0
dtype: float64

In [12]:
libros["Comunidad autónoma"].unique() #no todas las tablas tiene datos de Ceuta y Melilla

array(['Andalucía', 'Aragón', 'Asturias (Principado de)',
       'Balears (Illes)', 'Canarias', 'Cantabria', 'Castilla y León',
       'Castilla-La Mancha', 'Cataluña', 'Comunitat Valenciana',
       'Extremadura', 'Galicia', 'Madrid (Comunidad de)',
       'Murcia (Región de)', 'Navarra (Comunidad Foral de)', 'País Vasco',
       'Rioja (La)', 'Ceuta y Melilla'], dtype=object)

In [13]:
libros = libros[libros['Comunidad autónoma'] != "Ceuta y Melilla"]

In [14]:
libros["Indicador_libros"].unique()

array(['TOTAL (Millones de euros)', 'GASTO MEDIO POR HOGAR (Euros)',
       'GASTO MEDIO POR UNIDAD DE CONSUMO (Euros)',
       'GASTO MEDIO POR PERSONA (Euros)'], dtype=object)

In [15]:
libros = libros[libros['Indicador_libros'] != "GASTO MEDIO POR UNIDAD DE CONSUMO (Euros)"]

In [16]:
libros.info()

<class 'pandas.core.frame.DataFrame'>
Index: 306 entries, 0 to 1130
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   periodo                306 non-null    int64  
 1   Comunidad autónoma     306 non-null    object 
 2   Grupo de gasto_libros  306 non-null    object 
 3   Indicador_libros       306 non-null    object 
 4   Total_libros           306 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 14.3+ KB


In [17]:
libros.sort_values(by= ["periodo","Comunidad autónoma","Grupo de gasto_libros"], ascending=[False, True, True], inplace=True)
libros.head(10)

,periodo,Comunidad autónoma,Grupo de gasto_libros,Indicador_libros,Total_libros
0,2023,Andalucía,Libro,TOTAL (Millones de euros),181.5
288,2023,Andalucía,Libro,GASTO MEDIO POR HOGAR (Euros),54.0
864,2023,Andalucía,Libro,GASTO MEDIO POR PERSONA (Euros),21.2
8,2023,Andalucía,Publicaciones periódicas,TOTAL (Millones de euros),40.9
296,2023,Andalucía,Publicaciones periódicas,GASTO MEDIO POR HOGAR (Euros),12.2
872,2023,Andalucía,Publicaciones periódicas,GASTO MEDIO POR PERSONA (Euros),4.8
16,2023,Aragón,Libro,TOTAL (Millones de euros),60.3
304,2023,Aragón,Libro,GASTO MEDIO POR HOGAR (Euros),107.9
880,2023,Aragón,Libro,GASTO MEDIO POR PERSONA (Euros),45.3
24,2023,Aragón,Publicaciones periódicas,TOTAL (Millones de euros),18.4


## 🎭 Espectáculos

In [18]:
espectaculos.rename(columns={'Indicador': 'Indicador_espectaculos', 'Total': 'Total_espectaculos'}, inplace=True)
espectaculos = espectaculos[['periodo','Comunidad autónoma','Indicador_espectaculos', 'Total_espectaculos']]

In [19]:
espectaculos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   periodo                 576 non-null    int64 
 1   Comunidad autónoma      576 non-null    object
 2   Indicador_espectaculos  576 non-null    object
 3   Total_espectaculos      576 non-null    object
dtypes: int64(1), object(3)
memory usage: 18.1+ KB


In [20]:
# Reemplazar comas por puntos
espectaculos["Total_espectaculos"] = espectaculos["Total_espectaculos"].str.replace(',', '.')

# Convertir a numérico (y evitar errores cuando hay valores como '..' )
espectaculos["Total_espectaculos"] = pd.to_numeric(espectaculos["Total_espectaculos"], errors='coerce')

In [21]:
espectaculos.info() #Las columnas son del tipo correspondiente

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   periodo                 576 non-null    int64  
 1   Comunidad autónoma      576 non-null    object 
 2   Indicador_espectaculos  576 non-null    object 
 3   Total_espectaculos      544 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 18.1+ KB


In [22]:
espectaculos["periodo"].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016])

In [23]:
espectaculos = espectaculos[espectaculos["periodo"] >= 2021]

In [24]:
espectaculos["Comunidad autónoma"].unique()

array(['Andalucía', 'Aragón', 'Asturias (Principado de)',
       'Balears (Illes)', 'Canarias', 'Cantabria', 'Castilla y León',
       'Castilla-La Mancha', 'Cataluña', 'Comunitat Valenciana',
       'Extremadura', 'Galicia', 'Madrid (Comunidad de)',
       'Murcia (Región de)', 'Navarra (Comunidad Foral de)', 'País Vasco',
       'Rioja (La)', 'Ceuta y Melilla'], dtype=object)

In [25]:
espectaculos = espectaculos[espectaculos['Comunidad autónoma'] != "Ceuta y Melilla"]

In [26]:
espectaculos["Indicador_espectaculos"].unique()

array(['TOTAL (Millones de euros)', 'GASTO MEDIO POR HOGAR (Euros)',
       'GASTO MEDIO POR UNIDAD DE CONSUMO (Euros)',
       'GASTO MEDIO POR PERSONA (Euros)'], dtype=object)

In [27]:
espectaculos = espectaculos[espectaculos['Indicador_espectaculos'] != "GASTO MEDIO POR UNIDAD DE CONSUMO (Euros)"]

In [28]:
espectaculos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 153 entries, 0 to 562
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   periodo                 153 non-null    int64  
 1   Comunidad autónoma      153 non-null    object 
 2   Indicador_espectaculos  153 non-null    object 
 3   Total_espectaculos      153 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.0+ KB


In [29]:
espectaculos.sort_values(by= ["periodo","Comunidad autónoma"], ascending=[False, True], inplace=True)
espectaculos.head()

,periodo,Comunidad autónoma,Indicador_espectaculos,Total_espectaculos
0,2023,Andalucía,TOTAL (Millones de euros),231.5
144,2023,Andalucía,GASTO MEDIO POR HOGAR (Euros),68.8
432,2023,Andalucía,GASTO MEDIO POR PERSONA (Euros),27.0
8,2023,Aragón,TOTAL (Millones de euros),59.5
152,2023,Aragón,GASTO MEDIO POR HOGAR (Euros),106.4


## 🎓 Educación

In [30]:
educacion.rename(columns={ 'Total': 'Total_educacion'}, inplace=True)
educacion = educacion[['periodo','Comunidad autónoma', 'Sexo', 'Año', 'Nivel de formación alcanzado','Total_educacion']]

In [31]:
educacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3192 entries, 0 to 3191
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   periodo                       3192 non-null   object
 1   Comunidad autónoma            3192 non-null   object
 2   Sexo                          3192 non-null   object
 3   Año                           3192 non-null   object
 4   Nivel de formación alcanzado  3192 non-null   object
 5   Total_educacion               3192 non-null   object
dtypes: object(6)
memory usage: 149.8+ KB


In [32]:
# arreglar educacion para que solo haya un año haciendo la media entre los trimestres
educacion.head()

,periodo,Comunidad autónoma,Sexo,Año,Nivel de formación alcanzado,Total_educacion
0,2023,Andalucía,Hombres,2023T4,Analfabetos,"1,5"
1,2023,Andalucía,Hombres,2023T3,Analfabetos,"1,2"
2,2023,Andalucía,Hombres,2023T2,Analfabetos,"1,4"
3,2023,Andalucía,Hombres,2023T1,Analfabetos,"1,2"
4,2022,Andalucía,Hombres,2022T4,Analfabetos,"1,5"


In [33]:
#ARREGLAR
# Reemplazar comas por puntos
educacion["Total_educacion"] = educacion["Total_educacion"].str.replace(',', '.')

# Convertir a numérico (y evitar errores cuando hay valores como '..' )
educacion["Total_educacion"] = pd.to_numeric(educacion["Total_educacion"], errors='coerce')


In [34]:
educacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3192 entries, 0 to 3191
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   periodo                       3192 non-null   object 
 1   Comunidad autónoma            3192 non-null   object 
 2   Sexo                          3192 non-null   object 
 3   Año                           3192 non-null   object 
 4   Nivel de formación alcanzado  3192 non-null   object 
 5   Total_educacion               3188 non-null   float64
dtypes: float64(1), object(5)
memory usage: 149.8+ KB


In [35]:
educacion[educacion.Total_educacion.isnull()] #Estos NaN se borrarán al eliminar los datos de Ceuta 

,periodo,Comunidad autónoma,Sexo,Año,Nivel de formación alcanzado,Total_educacion
1435,2022,Ceuta,Hombres,2022T1,Analfabetos,NaN
1436,2021,Ceuta,Hombres,2021T4,Analfabetos,NaN
1437,2021,Ceuta,Hombres,2021T3,Analfabetos,NaN
1438,2021,Ceuta,Hombres,2021T2,Analfabetos,NaN


In [36]:
educacion.drop(columns='Año', inplace=True)
educacion.drop_duplicates(inplace=True)

In [37]:
total=[]
for año in educacion['periodo'].unique():
    educacion_año = educacion[educacion['periodo'] == año]
    media = educacion.groupby(['periodo', 'Comunidad autónoma', 'Sexo', 'Nivel de formación alcanzado'],as_index=False)['Total_educacion'].mean()
    total.append(media)

In [38]:
total

[    periodo Comunidad autónoma     Sexo  \
 0      2021          Andalucía  Hombres   
 1      2021          Andalucía  Hombres   
 2      2021          Andalucía  Hombres   
 3      2021          Andalucía  Hombres   
 4      2021          Andalucía  Hombres   
 ..      ...                ...      ...   
 793    2023          Rioja, La  Mujeres   
 794    2023          Rioja, La  Mujeres   
 795    2023          Rioja, La  Mujeres   
 796    2023          Rioja, La  Mujeres   
 797    2023          Rioja, La  Mujeres   
 
                           Nivel de formación alcanzado  Total_educacion  
 0                                          Analfabetos         1.900000  
 1                                   Educación primaria        10.875000  
 2                                   Educación superior        26.100000  
 3                       Estudios primarios incompletos         6.100000  
 4      Primera etapa de Educación Secundaria y similar        33.800000  
 ..                 

In [39]:
educacion = pd.concat(total, ignore_index=True)

In [40]:
educacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2394 entries, 0 to 2393
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   periodo                       2394 non-null   object 
 1   Comunidad autónoma            2394 non-null   object 
 2   Sexo                          2394 non-null   object 
 3   Nivel de formación alcanzado  2394 non-null   object 
 4   Total_educacion               2394 non-null   float64
dtypes: float64(1), object(4)
memory usage: 93.6+ KB


In [41]:
#periodo a int
educacion['periodo'] = pd.to_numeric(educacion['periodo'], errors='coerce')

In [42]:
educacion["periodo"].unique()

array([2021, 2022, 2023])

In [43]:
educacion["Comunidad autónoma"].unique() 

array(['Andalucía', 'Aragón', 'Asturias, Principado de', 'Balears, Illes',
       'Canarias', 'Cantabria', 'Castilla - La Mancha', 'Castilla y León',
       'Cataluña', 'Ceuta', 'Comunitat Valenciana', 'Extremadura',
       'Galicia', 'Madrid, Comunidad de', 'Melilla', 'Murcia, Región de',
       'Navarra, Comunidad Foral de', 'País Vasco', 'Rioja, La'],
      dtype=object)

In [44]:
def formatear_nombre(nombre):
    if "," in nombre:
        partes= nombre.split(",")
        partes= [a.strip() for a in partes] #para que no salga 'Asturias ( Principado de)'
        nombre_cambio = f"{partes[0]} ({partes[1]})"
        return nombre_cambio
    else:
        return nombre


In [45]:
educacion["Comunidad autónoma"] = educacion["Comunidad autónoma"].apply(formatear_nombre)

In [46]:
educacion["Comunidad autónoma"].unique()

array(['Andalucía', 'Aragón', 'Asturias (Principado de)',
       'Balears (Illes)', 'Canarias', 'Cantabria', 'Castilla - La Mancha',
       'Castilla y León', 'Cataluña', 'Ceuta', 'Comunitat Valenciana',
       'Extremadura', 'Galicia', 'Madrid (Comunidad de)', 'Melilla',
       'Murcia (Región de)', 'Navarra (Comunidad Foral de)', 'País Vasco',
       'Rioja (La)'], dtype=object)

In [47]:
educacion = educacion[educacion["Comunidad autónoma"] != "Ceuta"]
educacion = educacion[educacion["Comunidad autónoma"] != "Melilla"]

In [48]:
educacion.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2142 entries, 0 to 2393
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   periodo                       2142 non-null   int64  
 1   Comunidad autónoma            2142 non-null   object 
 2   Sexo                          2142 non-null   object 
 3   Nivel de formación alcanzado  2142 non-null   object 
 4   Total_educacion               2142 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 100.4+ KB


In [49]:
educacion.drop_duplicates(inplace=True)

In [50]:
educacion.info()

<class 'pandas.core.frame.DataFrame'>
Index: 714 entries, 0 to 797
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   periodo                       714 non-null    int64  
 1   Comunidad autónoma            714 non-null    object 
 2   Sexo                          714 non-null    object 
 3   Nivel de formación alcanzado  714 non-null    object 
 4   Total_educacion               714 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 33.5+ KB


In [51]:
educacion.sort_values(by= ["periodo","Comunidad autónoma","Sexo","Nivel de formación alcanzado"], ascending=[False, True, True, True], inplace=True)
educacion.head()

,periodo,Comunidad autónoma,Sexo,Nivel de formación alcanzado,Total_educacion
532,2023,Andalucía,Hombres,Analfabetos,1.366667
533,2023,Andalucía,Hombres,Educación primaria,10.650000
534,2023,Andalucía,Hombres,Educación superior,27.466667
535,2023,Andalucía,Hombres,Estudios primarios incompletos,5.600000
536,2023,Andalucía,Hombres,Primera etapa de Educación Secundaria y similar,33.175000


## 📰 Bienes y servicios

In [52]:
bys.head()
bys.rename(columns={'Indicador': 'Indicador_bys', 'Grupos de gasto': 'Grupo de gasto_bys', 'Total': 'Total_bys'}, inplace=True)
bys = bys[['periodo','Comunidad autónoma', 'Grupo de gasto_bys', 'Indicador_bys', 'Total_bys']]

In [53]:
bys.head()

,periodo,Comunidad autónoma,Grupo de gasto_bys,Indicador_bys,Total_bys
0,2023,Andalucía,Libro y publicaciones periódicas,TOTAL (Millones de euros),"222,4"
1,2022,Andalucía,Libro y publicaciones periódicas,TOTAL (Millones de euros),"184,9"
2,2021,Andalucía,Libro y publicaciones periódicas,TOTAL (Millones de euros),"239,2"
3,2020,Andalucía,Libro y publicaciones periódicas,TOTAL (Millones de euros),"186,3"
4,2019,Andalucía,Libro y publicaciones periódicas,TOTAL (Millones de euros),"209,6"


In [54]:
bys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2880 entries, 0 to 2879
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   periodo             2880 non-null   int64 
 1   Comunidad autónoma  2880 non-null   object
 2   Grupo de gasto_bys  2880 non-null   object
 3   Indicador_bys       2880 non-null   object
 4   Total_bys           2880 non-null   object
dtypes: int64(1), object(4)
memory usage: 112.6+ KB


In [55]:
# Reemplazar comas por puntos
bys["Total_bys"] = bys["Total_bys"].str.replace(',', '.')

# Convertir a numérico (y evitar errores cuando hay valores como '..' )
bys["Total_bys"] = pd.to_numeric(bys["Total_bys"], errors='coerce') #Nos aparecen NaN --> por ejemplo por "Unable to parse string "1.066.3" at position 31"
bys.dropna(inplace=True)

In [56]:
bys.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2719 entries, 0 to 2847
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   periodo             2719 non-null   int64  
 1   Comunidad autónoma  2719 non-null   object 
 2   Grupo de gasto_bys  2719 non-null   object 
 3   Indicador_bys       2719 non-null   object 
 4   Total_bys           2719 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 127.5+ KB


In [57]:
bys["periodo"].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016])

In [58]:
bys = bys[bys["periodo"] >= 2021]

In [59]:
bys["Comunidad autónoma"].unique() #No están Ceuta y Melilla

array(['Andalucía', 'Aragón', 'Asturias (Principado de)',
       'Balears (Illes)', 'Canarias', 'Cantabria', 'Castilla y León',
       'Castilla-La Mancha', 'Cataluña', 'Comunitat Valenciana',
       'Extremadura', 'Galicia', 'Madrid (Comunidad de)',
       'Murcia (Región de)', 'Navarra (Comunidad Foral de)', 'País Vasco',
       'Rioja (La)'], dtype=object)

In [60]:
bys['Indicador_bys'].unique()

array(['TOTAL (Millones de euros)', 'PORCENTAJE',
       'GASTO MEDIO POR HOGAR (Euros)',
       'GASTO MEDIO POR UNIDAD DE CONSUMO (Euros)',
       'GASTO MEDIO POR PERSONA (Euros)'], dtype=object)

In [61]:
bys = bys[bys['Indicador_bys'] != "GASTO MEDIO POR UNIDAD DE CONSUMO (Euros)"]
bys = bys[bys['Indicador_bys'] != "PORCENTAJE"]

In [62]:
bys['Grupo de gasto_bys'].unique()

array(['Libro y publicaciones periódicas', 'Servicios culturales',
       'Soportes, equipos y accesorios audiovisuales y de tratamiento de la información',
       'Telefonía móvil y servicios relacionados con Internet'],
      dtype=object)

In [63]:
bys = bys[bys['Grupo de gasto_bys'] != 'Telefonía móvil y servicios relacionados con Internet']
bys = bys[bys['Grupo de gasto_bys'] != 'Soportes, equipos y accesorios audiovisuales y de tratamiento de la información']

In [64]:
bys.info()

<class 'pandas.core.frame.DataFrame'>
Index: 306 entries, 0 to 2826
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   periodo             306 non-null    int64  
 1   Comunidad autónoma  306 non-null    object 
 2   Grupo de gasto_bys  306 non-null    object 
 3   Indicador_bys       306 non-null    object 
 4   Total_bys           306 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 14.3+ KB


In [65]:
bys.sort_values(by= ["periodo","Comunidad autónoma","Grupo de gasto_bys","Indicador_bys"], ascending=[False, True, True, True], inplace=True)
bys.head(10)

,periodo,Comunidad autónoma,Grupo de gasto_bys,Indicador_bys,Total_bys
1152,2023,Andalucía,Libro y publicaciones periódicas,GASTO MEDIO POR HOGAR (Euros),66.1
2304,2023,Andalucía,Libro y publicaciones periódicas,GASTO MEDIO POR PERSONA (Euros),26.0
0,2023,Andalucía,Libro y publicaciones periódicas,TOTAL (Millones de euros),222.4
1160,2023,Andalucía,Servicios culturales,GASTO MEDIO POR HOGAR (Euros),95.0
2312,2023,Andalucía,Servicios culturales,GASTO MEDIO POR PERSONA (Euros),37.3
8,2023,Andalucía,Servicios culturales,TOTAL (Millones de euros),319.7
1184,2023,Aragón,Libro y publicaciones periódicas,GASTO MEDIO POR HOGAR (Euros),140.8
2336,2023,Aragón,Libro y publicaciones periódicas,GASTO MEDIO POR PERSONA (Euros),59.1
32,2023,Aragón,Libro y publicaciones periódicas,TOTAL (Millones de euros),78.7
1192,2023,Aragón,Servicios culturales,GASTO MEDIO POR HOGAR (Euros),164.1


## 💶 Renta

In [78]:
renta.rename(columns={'Comunidades y Ciudades Autónomas': 'Comunidad autónoma','Renta anual neta media por persona y por unidad de consumo': 'Renta anual', 'Periodo':'periodo', 'Total': 'Total_renta'}, inplace=True)
renta = renta[['periodo', 'Comunidad autónoma', 'Renta anual', 'Total_renta']] 

In [79]:
renta.info() #los dtypes son correctos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   periodo             76 non-null     int64  
 1   Comunidad autónoma  76 non-null     object 
 2   Renta anual         76 non-null     object 
 3   Total_renta         76 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.5+ KB


In [80]:
renta = renta[renta["periodo"] >= 2021]
renta = renta[renta["periodo"] < 2024]

In [81]:
renta.periodo.unique()

array([2023, 2022, 2021])

In [82]:
renta['Total_renta'] = renta['Total_renta'] * 1000

In [83]:
renta["Comunidad autónoma"].unique()

array(['01 Andalucía', '02 Aragón', '03 Asturias, Principado de',
       '04 Balears, Illes', '05 Canarias', '06 Cantabria',
       '07 Castilla y León', '08 Castilla - La Mancha', '09 Cataluña',
       '10 Comunitat Valenciana', '11 Extremadura', '12 Galicia',
       '13 Madrid, Comunidad de', '14 Murcia, Región de',
       '15 Navarra, Comunidad Foral de', '16 País Vasco', '17 Rioja, La',
       '18 Ceuta', '19 Melilla'], dtype=object)

In [84]:
renta["Comunidad autónoma"] = renta["Comunidad autónoma"].apply(lambda x: x.split(" ", 1)[1])

In [85]:
renta.head()

,periodo,Comunidad autónoma,Renta anual,Total_renta
1,2023,Andalucía,Renta neta media por persona,11719.0
2,2022,Andalucía,Renta neta media por persona,10703.0
3,2021,Andalucía,Renta neta media por persona,9915.0
5,2023,Aragón,Renta neta media por persona,14810.0
6,2022,Aragón,Renta neta media por persona,14015.0


In [86]:
renta = renta[renta["Comunidad autónoma"] != "Ceuta"]
renta = renta[renta["Comunidad autónoma"] != "Melilla"]

In [87]:
renta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, 1 to 67
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   periodo             51 non-null     int64  
 1   Comunidad autónoma  51 non-null     object 
 2   Renta anual         51 non-null     object 
 3   Total_renta         51 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.0+ KB


In [88]:
renta.sort_values(by= ["periodo","Comunidad autónoma", "Renta anual", "Total_renta"], ascending=[False, True, False, False], inplace=True)
renta.head()

,periodo,Comunidad autónoma,Renta anual,Total_renta
1,2023,Andalucía,Renta neta media por persona,11719.0
5,2023,Aragón,Renta neta media por persona,14810.0
9,2023,"Asturias, Principado de",Renta neta media por persona,15432.0
13,2023,"Balears, Illes",Renta neta media por persona,14139.0
17,2023,Canarias,Renta neta media por persona,12177.0


## 💰 Ingresos

In [89]:
ingresos.rename(columns={'Indicador': 'Indicador_ingresos', 'Total': 'Total_ingresos'}, inplace=True)
ingresos = ingresos[['periodo', 'Indicador_ingresos', 'Ingresos mensuales en el hogar', 'Total_ingresos']] 

In [90]:
ingresos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   periodo                         280 non-null    int64 
 1   Indicador_ingresos              280 non-null    object
 2   Ingresos mensuales en el hogar  280 non-null    object
 3   Total_ingresos                  280 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.9+ KB


In [91]:
ingresos["Total_ingresos"] = ingresos["Total_ingresos"].str.replace(r'\.(?=[0-9]{3})', '', regex=True)  # Elimina los puntos de miles --> necesario para no tener NaN
ingresos["Total_ingresos"] = ingresos["Total_ingresos"].str.replace(',', '.')  # Reemplaza la coma por punto
ingresos['Total_ingresos'] = pd.to_numeric(ingresos['Total_ingresos'], errors='coerce')  # Convierte a numérico

In [92]:
ingresos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   periodo                         280 non-null    int64  
 1   Indicador_ingresos              280 non-null    object 
 2   Ingresos mensuales en el hogar  280 non-null    object 
 3   Total_ingresos                  280 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 8.9+ KB


In [93]:
ingresos.periodo.unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016])

In [94]:
ingresos = ingresos[ingresos.periodo >=2021]

In [95]:
ingresos["Indicador_ingresos"].unique()

array(['TOTAL (Millones de euros)',
       'EN PORCENTAJE DEL GASTO EN EL TOTAL DE BIENES Y SERVICIOS',
       'GASTO MEDIO POR HOGAR (Euros)',
       'GASTO MEDIO POR UNIDAD DE CONSUMO (Euros)',
       'GASTO MEDIO POR PERSONA (Euros)'], dtype=object)

In [96]:
ingresos = ingresos[ingresos["Indicador_ingresos"] != "GASTO MEDIO POR UNIDAD DE CONSUMO (Euros)"]

In [97]:
ingresos.sort_values(by= ["periodo", "Indicador_ingresos", "Ingresos mensuales en el hogar"], ascending=[False, False, False], inplace=True)
ingresos.head()

,periodo,Indicador_ingresos,Ingresos mensuales en el hogar,Total_ingresos
0,2023,TOTAL (Millones de euros),Total,11306.9
8,2023,TOTAL (Millones de euros),Menos de 1.000 euros,493.6
40,2023,TOTAL (Millones de euros),Entre 2.500 y 2.999 euros,1887.2
32,2023,TOTAL (Millones de euros),Entre 2.000 y 2.499 euros,1437.6
24,2023,TOTAL (Millones de euros),Entre 1.500 y 1.999 euros,1286.0


## 📊 Características

In [98]:
caracteristicas.rename(columns={'Indicador': 'Indicador_caracteristicas', 'Total': 'Total_caracteristicas'}, inplace=True)
caracteristicas = caracteristicas[['periodo', 'Indicador_caracteristicas', 'Características del sustentador principal', 'Total_caracteristicas']] 

In [99]:
caracteristicas["Total_caracteristicas"] = caracteristicas["Total_caracteristicas"].str.replace(r'\.(?=\d{3})', '', regex=True)  # Elimina los puntos de miles --> necesario para no tener NaN
caracteristicas["Total_caracteristicas"] = caracteristicas["Total_caracteristicas"].str.replace(',', '.')

# Convertir a numérico (y evitar errores cuando hay valores como '..' )
caracteristicas["Total_caracteristicas"] = pd.to_numeric(caracteristicas["Total_caracteristicas"], errors = "coerce")

In [100]:
caracteristicas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   periodo                                    1000 non-null   int64  
 1   Indicador_caracteristicas                  1000 non-null   object 
 2   Características del sustentador principal  1000 non-null   object 
 3   Total_caracteristicas                      1000 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 31.4+ KB


In [101]:
caracteristicas.periodo.unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016])

In [102]:
caracteristicas = caracteristicas[caracteristicas.periodo >=2021]

In [103]:
caracteristicas.Indicador_caracteristicas.unique()

array(['TOTAL (Millones de euros)',
       'EN PORCENTAJE DEL GASTO EN EL TOTAL DE BIENES Y SERVICIOS',
       'GASTO MEDIO POR HOGAR (Euros)',
       'GASTO MEDIO POR UNIDAD DE CONSUMO (Euros)',
       'GASTO MEDIO POR PERSONA (Euros)'], dtype=object)

In [104]:
caracteristicas = caracteristicas[caracteristicas["Indicador_caracteristicas"] != "GASTO MEDIO POR UNIDAD DE CONSUMO (Euros)"]

In [105]:
caracteristicas.sort_values(by= ["periodo", "Indicador_caracteristicas", "Características del sustentador principal"], ascending=[False, False, False], inplace=True)
caracteristicas.head()

,periodo,Indicador_caracteristicas,Características del sustentador principal,Total_caracteristicas
184,2023,TOTAL (Millones de euros),Tipo de hogar: Un adulto con hijos,864.9
144,2023,TOTAL (Millones de euros),Tipo de hogar: Persona sola,1990.3
152,2023,TOTAL (Millones de euros),Tipo de hogar: Pareja sin hijos,2221.6
160,2023,TOTAL (Millones de euros),Tipo de hogar: Pareja con un hijo,2023.9
176,2023,TOTAL (Millones de euros),Tipo de hogar: Pareja con tres o más hijos,851.3


### Guardamos los datos

In [ ]:
with pd.ExcelWriter("../data/datos_tratados.xlsx", engine="openpyxl") as writer:
    libros.to_excel(writer, sheet_name="libros", index=False)
    espectaculos.to_excel(writer, sheet_name="espectaculos", index=False)
    educacion.to_excel(writer, sheet_name="educacion", index=False)
    bys.to_excel(writer, sheet_name="bys", index=False)
    renta.to_excel(writer, sheet_name="renta", index=False)
    ingresos.to_excel(writer, sheet_name="ingresos", index=False)
    caracteristicas.to_excel(writer, sheet_name="caracteristicas", index=False)